In [1]:
import json

data = json.load(open("dataset_600.json", 'r'))
print(data[0])

{'input': "What's the current weather in Dhaka?", 'output': 'TRUE: Current weather in Dhaka'}


In [2]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [3]:
import torch

device = "CUDA" if torch.cuda.is_available() else "CPU"
print(f"USING DEVICE {device}")

if device == "CUDA":
  print("USING", torch.cuda.get_device_name() )

USING DEVICE CUDA
USING Tesla T4


In [4]:
import unsloth
from unsloth import FastLanguageModel
unsloth.WANDB_DISABLED = True
model_name = "unsloth/SmolLM-135M"
max_seq_length = 512
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/112M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/978 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in data]
dataset = Dataset.from_dict({"text": formatted_data})

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.8.10 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
    ),
)
trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/600 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 3 | Total steps = 225
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,537,920 of 154,052,928 (12.68% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammedtahmid-bd (mohammedtahmid-bd-wertg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,entropy
25,1.878200,0
50,0.254300,No Log
75,0.182800,No Log
100,0.162800,No Log
125,0.169700,No Log
150,0.159200,No Log
175,0.155000,No Log
200,0.155300,No Log
225,0.156500,No Log


In [8]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(49152, 576, padding_idx=16)
        (layers): ModuleList(
          (0-29): 30 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=576, out_features=576, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=576, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=576, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [9]:
# Re-using the format_prompt function defined earlier
def format_prompt(example):
    return f"### Input: {example['input']}\n### Output:"

messages = [
    {"input": "open up firefox"},
]
# Apply the custom formatting to the input
formatted_input = format_prompt(messages[0])

inputs = tokenizer(
    formatted_input,
    tokenize=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)
response = tokenizer.batch_decode(outputs)[0]
print(response)

TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'tokenize'

You can save the model and tokenizer locally:

In [ ]:
model.save_pretrained("finetuned_smollm")
tokenizer.save_pretrained("finetuned_smollm")

Or push them to the Hugging Face Hub (replace "your_username" with your actual username):

In [ ]:
# from huggingface_hub import login
# login() # You'll be prompted to enter your Hugging Face token

# model.push_to_hub("your_username/finetuned_smollm")
# tokenizer.push_to_hub("your_username/finetuned_smollm")

To load the saved model and tokenizer for inference:

In [ ]:
from unsloth import FastLanguageModel

# Load the model and tokenizer from the saved directory
model_path = "finetuned_smollm" # Replace with the path where you saved your model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path, # Load from the local path
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

# You can also load the model for inference directly
FastLanguageModel.for_inference(model)

Now you can use the loaded model and tokenizer for inference, just like before.

In [ ]:
# Re-using the format_prompt function defined earlier
def format_prompt(example):
    return f"### Input: {example['input']}\n### Output:"

messages = [
    {"input": "if 2x = 6+9 then solve for x"},
]
# Apply the custom formatting to the input
formatted_input = format_prompt(messages[0])

inputs = tokenizer(
    formatted_input,
    return_tensors="pt",
    padding=True, # Add padding
    truncation=True, # Add truncation
    max_length=max_seq_length, # Specify max_length
).to("cuda")

outputs = model.generate(
    input_ids=inputs["input_ids"], # Explicitly pass input_ids
    attention_mask=inputs["attention_mask"], # Explicitly pass attention_mask
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)
response = tokenizer.batch_decode(outputs)[0]
print(response)

# Task
Rewrite the provided Python code for finetuning a language model using PEFT and SFTTrainer, making it cleaner and more organized. The rewritten code should include sections for setup and data loading, model loading, data preparation, model training, inference, and saving/loading the finetuned model.

## Setup and data loading

### Subtask:
Combine necessary imports and data loading into a single section.


**Reasoning**:
Combine all the necessary imports and data loading steps into a single code block as per the instructions.



In [ ]:
import json
import torch

data = json.load(open("dataset_600.json", 'r'))
print(data[0])

device = "CUDA" if torch.cuda.is_available() else "CPU"
print(f"USING DEVICE {device}")

if device == "CUDA":
  print("USING", torch.cuda.get_device_name() )

## Model loading

### Subtask:
Load the pre-trained model and tokenizer.


**Reasoning**:
Load the pre-trained model and tokenizer using FastLanguageModel.from_pretrained with the specified parameters, including 4-bit quantization.



In [10]:
import unsloth
from unsloth import FastLanguageModel

unsloth.WANDB_DISABLED = True
model_name = "unsloth/SmolLM-135M"
max_seq_length = 512
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True)

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Data preparation

### Subtask:
Format the data for training and create a Hugging Face Dataset.


**Reasoning**:
The subtask requires formatting the data and creating a Hugging Face Dataset. This involves defining a formatting function, applying it to the data, and then creating the dataset.



In [11]:
from datasets import Dataset
import json

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in data]
dataset = Dataset.from_dict({"text": formatted_data})
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 600
})


## Model training

### Subtask:
Define the PEFT model and set up the SFTTrainer for finetuning.


**Reasoning**:
Define the PEFT model and set up the SFTTrainer for finetuning.



In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments

model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=25,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    save_strategy="epoch",
    save_total_limit=2,
    dataloader_pin_memory=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=training_args,
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/600 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 600 | Num Epochs = 3 | Total steps = 225
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,537,920 of 154,052,928 (12.68% trained)


Step,Training Loss,entropy
25,1.878200,0
50,0.254300,No Log
75,0.182800,No Log
100,0.162800,No Log
125,0.169700,No Log
150,0.159200,No Log
175,0.155000,No Log
200,0.155300,No Log
225,0.156500,No Log


## Inference

### Subtask:
Prepare the model for inference and demonstrate how to generate responses.


**Reasoning**:
Prepare the model for inference, define the prompt formatting function, create input messages, format the input, tokenize it, generate a response from the model, decode the response, and print it.



In [13]:
FastLanguageModel.for_inference(model)

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output:"

messages = [
    {"input": "if 2x = 6+9 then solve for x"},
    {"input": "What's the current weather in London?"}
]

for message in messages:
    formatted_input = format_prompt(message)

    inputs = tokenizer(
        formatted_input,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_seq_length,
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=256,
        use_cache=True,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )
    response = tokenizer.batch_decode(outputs)[0]
    print(response)

### Input: if 2x = 6+9 then solve for x
### Output: "TRUE: "<|endoftext|>
### Input: What's the current weather in London?
### Output: "TRUE: Current weather in London"<|endoftext|>


## Saving and loading

### Subtask:
Include steps for saving and loading the finetuned model and tokenizer.


**Reasoning**:
Save the finetuned model and tokenizer to a local directory.



In [14]:
model_save_path = "finetuned_smollm"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to finetuned_smollm


**Reasoning**:
Load the saved model and tokenizer and prepare the model for inference.



In [15]:
from unsloth import FastLanguageModel

model_path = "finetuned_smollm"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
print(f"Model and tokenizer loaded from {model_path} and prepared for inference.")

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model and tokenizer loaded from finetuned_smollm and prepared for inference.


## Summary:

### Data Analysis Key Findings

*   The script successfully loaded data from a JSON file, identified the available device (CUDA or CPU), and loaded the pre-trained `unsloth/SmolLM-135M` model and tokenizer, utilizing 4-bit quantization.
*   The data was correctly formatted into a prompt structure (`### Input: ...\n### Output: ...<|endoftext|>`) and converted into a Hugging Face Dataset with a "text" column.
*   The model was configured for PEFT training using LoRA with specified parameters ($\text{r}=64$, $\text{lora\_alpha}=128$), and the SFTTrainer was successfully set up with defined training arguments (e.g., batch size 2, 3 epochs, learning rate $\text{2e}^{-4}$).
*   The training process was initiated and ran successfully.
*   The finetuned model was prepared for inference, and the inference process was demonstrated by generating responses for example prompts.
*   The finetuned model and tokenizer were successfully saved to a local directory and subsequently loaded back for inference.

### Insights or Next Steps

*   The modular approach of separating the code into distinct sections (Setup/Data Loading, Model Loading, Data Preparation, Training, Inference, Saving/Loading) significantly improves code readability and organization, making it easier to understand and maintain the finetuning pipeline.
*   The saved model and tokenizer can be easily deployed for inference or further fine-tuning, providing a clear pathway for utilizing the trained model in other applications.


You can save the model and tokenizer locally:

In [18]:
model.save_pretrained_gguf("pwd", tokenizer, quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 111.9M


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.3 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 30/30 [00:00<00:00, 79.87it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving pwd/pytorch_model.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at pwd into f16 GGUF format.
The output location will be /content/pwd/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: pwd
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {576, 49152}
INFO:hf-to-gguf:blk.0.attn_q.weight,     

Or push them to the Hugging Face Hub (replace "your_username" with your actual username):

In [17]:
# from huggingface_hub import login
# login() # You'll be prompted to enter your Hugging Face token
print("hello world")